In [1]:
import pandas as pd
news_outlets_ph_reports = pd.read_csv("../data/bquxjob_5566f61b_18bbbf32142.csv").dropna()

In [113]:
# Source: https://www.statista.com/statistics/1219827/philippines-trust-in-news-outlets-by-brand/
# tv5, DZRH not monitored by GDELT
# DZBB owned by gmanews, DZMM Teleradyo owned by abscbn news
most_trusted_news_outlets = ["gmanews", "philstar", "tv5", "mb\.com", "\.inquirer\.net", "dzrh", "abs-cbn"]

In [121]:
# Look for outlets using subdomains in urls
outlets_using_subdomains = ["\.philstar\.com", "\.inquirer\.net", "\.rappler\.com"]
mask = news_outlets_ph_reports["host_url"].str.extract(rf'({"|".join(outlets_using_subdomains)})', expand=False).isna()==False
news_outlets_ph_reports[mask]

,host_url,tot_phl_relevant,avg_phl_relevant,grouped,grouped_host_url
172,bandera.inquirer.net,37,0.345794,.inquirer.net,.inquirer.net
213,www.inquirer.net,26,0.619048,.inquirer.net,.inquirer.net
266,pep.inquirer.net,47,0.626667,.inquirer.net,.inquirer.net
543,newsinfo.inquirer.net,86630,0.420098,.inquirer.net,.inquirer.net
777,globalnation.inquirer.net,14258,0.546242,.inquirer.net,.inquirer.net
789,scoutmediaph.rappler.com,9,0.290323,rappler,rappler
1604,cebudailynews.inquirer.net,18959,0.632114,.inquirer.net,.inquirer.net
1857,usa.inquirer.net,3680,0.400218,.inquirer.net,.inquirer.net
1890,entertainment.inquirer.net,1755,0.372928,.inquirer.net,.inquirer.net
2172,opinion.inquirer.net,5025,0.381231,.inquirer.net,.inquirer.net


In [122]:
# Aggregate metrics
news_outlets_ph_reports["grouped_host_url"] = news_outlets_ph_reports["host_url"].str.extract(rf'({"|".join(outlets_using_subdomains)})', expand=False).fillna(news_outlets_ph_reports["host_url"])

grouped_news_outlets_ph_reports = pd.DataFrame(
    {
        "tot_phl_relevant":news_outlets_ph_reports.groupby("grouped_host_url")["tot_phl_relevant"].sum(), 
        "avg_phl_relevant":news_outlets_ph_reports.groupby("grouped_host_url").apply(lambda ser: ser["tot_phl_relevant"].sum() / ser["tot_phl_relevant"].div(ser["avg_phl_relevant"]).sum()).fillna(0)
    }
)

grouped_news_outlets_ph_reports.head(10)

/var/folders/4l/jbwb4mc545x6m_zf23gyjhm00000gn/T/ipykernel_75151/2133185300.py:7: RuntimeWarning: invalid value encountered in scalar divide
  "avg_phl_relevant":news_outlets_ph_reports.groupby("grouped_host_url").apply(lambda ser: ser["tot_phl_relevant"].sum() / ser["tot_phl_relevant"].div(ser["avg_phl_relevant"]).sum()).fillna(0)


,tot_phl_relevant,avg_phl_relevant
grouped_host_url,,
.inquirer.net,138240,0.446173
.philstar.com,80818,0.490823
.rappler.com,19630,0.459837
0110.be,0,0.000000
02elf.net,0,0.000000
0dm.tamddon.com,0,0.000000
0ver.org,0,0.000000
0x7f.dev,0,0.000000
1-a1072.azureedge.net,0,0.000000


In [123]:
# Limited to 12 (excluding msn because they're not PH-focused)
min_reports = 10*((365*3)-51)  # At least 10 a day, data queried Nov 10, 2023
grouped_news_outlets_ph_reports.query(f"(tot_phl_relevant > {min_reports})").sort_values("avg_phl_relevant", ascending=False)

,tot_phl_relevant,avg_phl_relevant
grouped_host_url,,
www.sunstar.com.ph,34505,0.592279
www.philippinetimes.com,24424,0.543420
www.cnnphilippines.com,21484,0.505887
.philstar.com,80818,0.490823
.rappler.com,19630,0.459837
www.gmanetwork.com,16696,0.459679
mb.com.ph,100426,0.458116
.inquirer.net,138240,0.446173
www.manilatimes.net,56629,0.400848


In [128]:
"','".join(grouped_news_outlets_ph_reports.query(f"(tot_phl_relevant > {min_reports})").sort_values("avg_phl_relevant", ascending=False).index.tolist()[:-1])

"www.sunstar.com.ph','www.philippinetimes.com','www.cnnphilippines.com','.philstar.com','.rappler.com','www.gmanetwork.com','mb.com.ph','.inquirer.net','www.manilatimes.net','tribune.net.ph','www.bworldonline.com','businessmirror.com.ph"